In [1]:
import pandas as pd
from keys import *
from IPython.display import JSON
import requests
from numpy import nan
from bs4 import BeautifulSoup
import re

## Import Aircrafts Data from csv with Capacity

from: http://www.lsv.fr/~sirangel/teaching/dataset/index.html


Name:	Name of the aircraft. \
ICAO:	4-letter ICAO code, if available. \
IATA:	3-letter IATA code (identifier). \
Capacity:	Seating capacity (empty for cargo aircrafts). \
Country:	Country or territory where aircraft maker is incorporated. - dropped - not needed


In [2]:
aircraft = pd.read_csv('aircrafts.csv', sep=',')

In [3]:
# drop column 'Country' because it is not needed
aircraft.drop('Country', axis=1, inplace=True)
aircraft.columns = ['aircraft_name', 'icao_aircraft', 'iata_aircraft','capacity']

In [4]:
aircraft.head(20)

,aircraft_name,icao_aircraft,iata_aircraft,capacity
0,Aerospatiale/Alenia ATR 42-300 / 320,AT43,AT4,50
1,Aerospatiale/Alenia ATR 42-500,AT45,AT5,50
2,Aerospatiale/Alenia ATR 42/ ATR 72,\N,ATR,74
3,Aerospatiale/Alenia ATR 72,AT72,AT7,74
4,Aerospatiale/BAC Concorde,CONC,SSC,128
5,Airbus A300 pax,A30B,AB3,200
6,Airbus A300-600 pax,A306,AB6,266
7,Airbus A310 Freighter,A310,31F,\N
8,Airbus A310 all pax models,A310,310,198
9,Airbus A310-200 Freighter,A310,31X,\N


In [5]:
#drop duplicated values in name, only two and not important for dataset
aircraft.drop_duplicates(subset='aircraft_name', inplace=True)

In [6]:
#clean dataset from some values
aircraft['capacity'] = aircraft['capacity'].replace('\\N', '0')
aircraft['aircraft_name'] = aircraft['aircraft_name'].str.replace(' pax', '')

In [7]:
aircraft['capacity'] = pd.to_numeric(aircraft['capacity'])

In [8]:
aircraft.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 216 entries, 0 to 217
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   aircraft_name  216 non-null    object
 1   icao_aircraft  216 non-null    object
 2   iata_aircraft  216 non-null    object
 3   capacity       216 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 8.4+ KB


Unfortunately the aircraft_name in the Aircraft dataset does not 100% match the aircraft name gathered from the AeroDataBox API. I will not import this Dataframe into my Database. \
The reason for using this dataset in the first place was to get a rough number of passengers for the flights based on the available capacity of tehe aircraft. However, capacity also changse between different airlines and also would not reflect the actual number of passengers. \
Data about aircraft capacity could also be gathered from the AeroDataBox API, but unfortunately there is a limit of 200 requests per Month and thus it won't be possible to get the data for the capacity for each flight with the free subscription plan.

## Gather Weather Data from OpenWeatherMaps API

Get weather forecast from OpenWeatherMaps for next 5 days in 3h interval \
api_key is saved in keys.py and loaded in the beginning with all other packages

1. Use pyowm package
2. Directly request the API


In [85]:
city = 'Berlin'
country = 'DE'

weather = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en')


JSON(weather.json())

<IPython.core.display.JSON object>

In [87]:
weather.json()

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1638457200,
   'main': {'temp': 3.55,
    'feels_like': -0.58,
    'temp_min': 1.54,
    'temp_max': 3.55,
    'pressure': 988,
    'sea_level': 988,
    'grnd_level': 994,
    'humidity': 66,
    'temp_kf': 2.01},
   'weather': [{'id': 801,
     'main': 'Clouds',
     'description': 'few clouds',
     'icon': '02n'}],
   'clouds': {'all': 20},
   'wind': {'speed': 5.21, 'deg': 289, 'gust': 10.04},
   'visibility': 10000,
   'pop': 0.02,
   'sys': {'pod': 'n'},
   'dt_txt': '2021-12-02 15:00:00'},
  {'dt': 1638468000,
   'main': {'temp': 2.63,
    'feels_like': -1,
    'temp_min': 0.8,
    'temp_max': 2.63,
    'pressure': 993,
    'sea_level': 993,
    'grnd_level': 997,
    'humidity': 71,
    'temp_kf': 1.83},
   'weather': [{'id': 802,
     'main': 'Clouds',
     'description': 'scattered clouds',
     'icon': '03n'}],
   'clouds': {'all': 27},
   'wind': {'speed': 3.91, 'deg': 300, 'gust': 8.6},
   'visibility': 10000,
   

In [5]:
def get_weather(weather):
    
    forecast_api = weather.json()['list']

    weather_info = []

    for forecast_3h in forecast_api: 
        weather_hour = {}
        weather_hour['weather_datetime'] = forecast_3h['dt_txt']
        weather_hour['main_weather'] = forecast_3h['weather'][0]['main']
        weather_hour['weather_detail'] = forecast_3h['weather'][0]['description']
        weather_hour['temperature'] = forecast_3h['main']['temp']
        weather_hour['feels_like'] = forecast_3h['main']['feels_like']
        weather_hour['humidity'] = forecast_3h['main']['humidity']
        weather_hour['wind'] = forecast_3h['wind']['speed']
        try: weather_hour['prob_perc'] = float(forecast_3h['pop'])
        except: weather_hour['prob_perc'] = 0
        try: weather_hour['rain_qty'] = float(forecast_3h['rain']['3h'])
        except: weather_hour['rain_qty'] = 0
        try: weather_hour['snow'] = float(forecast_3h['snow']['3h'])
        except: weather_hour['snow'] = 0
        weather_hour['municipality_iso_country'] = city + ',' + country
        weather_info.append(weather_hour)
    
    return weather_info
    
weather_data = pd.DataFrame(get_weather(weather))
weather_data.head()

,weather_datetime,main_weather,weather_detail,temperature,feels_like,humidity,wind,prob_perc,rain_qty,snow,municipality_iso_country
0,2021-11-30 15:00:00,Rain,light rain,1.14,-5.86,86,11.65,1.00,0.93,0.0,"Munich,DE"
1,2021-11-30 18:00:00,Rain,light rain,1.64,-5.18,89,11.09,1.00,1.01,0.0,"Munich,DE"
2,2021-11-30 21:00:00,Rain,light rain,2.41,-3.84,92,9.99,1.00,1.71,0.0,"Munich,DE"
3,2021-12-01 00:00:00,Rain,light rain,4.05,-1.44,92,9.19,1.00,1.73,0.0,"Munich,DE"
4,2021-12-01 03:00:00,Rain,light rain,4.76,0.01,90,7.55,0.85,0.67,0.0,"Munich,DE"


In [17]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   weather_datetime          40 non-null     object 
 1   main_weather              40 non-null     object 
 2   weather_detail            40 non-null     object 
 3   temperature               40 non-null     float64
 4   feels_like                40 non-null     float64
 5   humidity                  40 non-null     int64  
 6   wind                      40 non-null     float64
 7   prob_perc                 40 non-null     float64
 8   rain_qty                  40 non-null     float64
 9   snow                      40 non-null     float64
 10  municipality_iso_country  40 non-null     object 
dtypes: float64(6), int64(1), object(4)
memory usage: 3.6+ KB


## Get Flight Data from AeroDataBox through RapidAPI

Retrieve flight arrivals for a city for the next day \
api_key is again saved in keys.py and loaded at the beginning of the script

In [79]:
#be careful not to run too often as there is a maximum of 200 api calls per month
def get_flight_data(icao):
    today = datetime.date.today()
    tomorrow = str(today + datetime.timedelta(days=1))
    
    hours = [["00:00","11:59"],["12:00","23:59"]]
    flight_data = []
    for time in hours:
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"

        querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}

        headers = {
            'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
            'x-rapidapi-key': flights_key
            }

        flight_data.append(requests.request("GET", url, headers=headers, params=querystring).json())
    return flight_data

In [80]:
JSON(flight_data.json())

<IPython.core.display.JSON object>

In [ ]:
flight_data = get_flight_data('EDDB')

In [84]:
icao = 'EDDB'

def flight_detail_extraction(flights):
    
    try: terminal = flights['arrival']['terminal']
    except: terminal = None
    
    try: aircraft = flights['aircraft']['model']
    except: aircraft = None    
    
    flights_df = pd.json_normalize({
        'scheduled_arrival_time': flights['arrival']['scheduledTimeLocal'],
        'flight_number': flights['number'],
        'dep_airport' : flights['departure']['airport']['name'],
        'terminal' : terminal,
        'airline' : flights['airline']['name'],
        'aircraft' : aircraft,
        'icao_code' : icao
    })
    return flights_df 

def create_flight_df(flight_data):
    flight_arrivals = []

    for j in flight_data: 
        new_flights = pd.concat([flight_detail_extraction(flight) for flight in j['arrivals']])
        flight_arrivals.append(new_flights)

    flight_arrivals = pd.concat(flight_arrivals) 
    return flight_arrivals

   scheduled_arrival_time flight_number dep_airport terminal  airline  \
0  2021-12-02 09:45+01:00       U2 5698      Aarhus        1  easyJet   

      aircraft icao_code  
0  Airbus A319      EDDB  
   scheduled_arrival_time flight_number dep_airport terminal     airline  \
0  2021-12-02 14:25+01:00        BT 821      Tallin        1  Air Baltic   

          aircraft icao_code  
0  Airbus A220-300      EDDB  
   scheduled_arrival_time flight_number dep_airport terminal   airline  \
0  2021-12-02 14:20+01:00        OS 229      Vienna        1  Austrian   

      aircraft icao_code  
0  Airbus A321      EDDB  
   scheduled_arrival_time flight_number  dep_airport terminal    airline  \
0  2021-12-02 17:55+01:00       EW 8045  Duesseldorf        1  Eurowings   

      aircraft icao_code  
0  Airbus A320      EDDB  
   scheduled_arrival_time flight_number dep_airport terminal    airline  \
0  2021-12-02 17:40+01:00       LH 1946      Munich        1  Lufthansa   

            aircraft ic

In [82]:
flight_arrivals = create_flight_df(flight_data)
flight_arrivals.head()

,scheduled_arrival_time,flight_number,dep_airport,terminal,airline,aircraft,icao_code
0,2021-12-02 09:45+01:00,U2 5698,Aarhus,1,easyJet,Airbus A319,EDDB
0,2021-12-02 14:25+01:00,BT 821,Tallin,1,Air Baltic,Airbus A220-300,EDDB
0,2021-12-02 14:20+01:00,OS 229,Vienna,1,Austrian,Airbus A321,EDDB
0,2021-12-02 17:55+01:00,EW 8045,Duesseldorf,1,Eurowings,Airbus A320,EDDB
0,2021-12-02 17:40+01:00,LH 1946,Munich,1,Lufthansa,Bombardier CRJ900,EDDB


In [17]:
flight_arrivals.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102 entries, 0 to 0
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Scheduled_arrival_time  102 non-null    object
 1   flight_number           102 non-null    object
 2   from                    102 non-null    object
 3   terminal                74 non-null     object
 4   airline                 102 non-null    object
 5   aircraft                102 non-null    object
 6   icao_code               102 non-null    object
dtypes: object(7)
memory usage: 6.4+ KB


## Get Population Data for Cities from csv File or Wikipedia

1. Get city details from csv file
    - from: https://simplemaps.com/data/world-cities
2. Get city details from DeoDB Cities API through RapidAPI
    - to get city details one needs the wikidata ID
    - get the wikidata ID for a city through webscraping of wikidatalink on wikipedia page
2. Get city details by scraping Wikipedia

In [5]:
# read a csv file with city population info
city_pop = pd.read_csv('worldcities.csv', sep=',')
# add a column with the info of the city + country code to have a reference column for the database
city_pop = city_pop.assign(municipality_iso_country = lambda x: x['city'] + ',' + x['iso2'])

In [6]:
#drop some of the columns that are not needed
city_pop.drop(['city_ascii', 'iso3', 'admin_name','capital','id'], axis=1, inplace=True)

In [7]:
city_pop.head()

,city,lat,lng,country,iso2,population,municipality_iso_country
0,Tokyo,35.6897,139.6922,Japan,JP,37977000.0,"Tokyo,JP"
1,Jakarta,-6.2146,106.8451,Indonesia,ID,34540000.0,"Jakarta,ID"
2,Delhi,28.6600,77.2300,India,IN,29617000.0,"Delhi,IN"
3,Mumbai,18.9667,72.8333,India,IN,23355000.0,"Mumbai,IN"
4,Manila,14.6000,120.9833,Philippines,PH,23088000.0,"Manila,PH"


In [8]:
city_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41001 entries, 0 to 41000
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   city                      41001 non-null  object 
 1   lat                       41001 non-null  float64
 2   lng                       41001 non-null  float64
 3   country                   41001 non-null  object 
 4   iso2                      40970 non-null  object 
 5   population                40263 non-null  float64
 6   municipality_iso_country  40970 non-null  object 
dtypes: float64(3), object(4)
memory usage: 2.2+ MB


In [9]:
#test for unique values in the municipality_iso_country column
city_pop['municipality_iso_country'].nunique()

38588

In [10]:
#drop duplictes in municipality_iso_countrs table
city_pop.drop_duplicates(subset='municipality_iso_country', inplace=True)

In [9]:
# List of cities from which to fetch the population data
city_list = ['Berlin', 'Paris', 'Barcelona', 'Madrid', 'Istanbul', 'Moscow', 'London', 'Saint Petersburg', 'Kyiv', 'Rome', 'Bucharest', 'Minsk', 'Vienna', 'Hamburg', 'Warsaw', 'Budapest', 'Munich', 'Milan', 'Belgrade', 'Prague', 'Sofia', 'Birmingham', 'Cologne', 'Odessa', 'Stockholm', 'Naples', 'Turin', 'Amsterdam', 'Marseille', 'Zagreb', 'Copenhagen', 'Valencia', 'Krakow', 'Frankfurt', 'Zaragoza', 'Athens', 'Palermo', 'Helsinki', 'Rotterdam', 'Stuttgart', 'Riga', 'Düsseldorf', 'Vilnius', 'Leipzig', 'Dublin', 'Tallinn']

In [10]:
# get the city id from the wikipedia site of the city - there from the link to the wikidata page
def get_city_id(soup):
    
    if soup.find('li', {'id':'t-wikibase'}).find('a')['href'] != None:
        wikidata_link = soup.find('li', {'id':'t-wikibase'}).find('a')['href']
        city_id = re.search('Q\d+', wikidata_link).group()
        
    return city_id

# request city ids from wikipedia city page and create a list of all wikidata city ids
list_cityid = []
for city in city_list:
    url = 'https://en.wikipedia.org/wiki/{}'.format(city)
    web = requests.get(url, 'html.parser')
    soup = BeautifulSoup(web.content)
    list_cityid.append(get_city_id(soup))

In [11]:
list_cityid

['Q64',
 'Q90',
 'Q1492',
 'Q2807',
 'Q406',
 'Q649',
 'Q84',
 'Q656',
 'Q1899',
 'Q220',
 'Q19660',
 'Q2280',
 'Q1741',
 'Q1055',
 'Q270',
 'Q1781',
 'Q1726',
 'Q490',
 'Q3711',
 'Q1085',
 'Q472',
 'Q2256',
 'Q365',
 'Q1874',
 'Q1754',
 'Q2634',
 'Q495',
 'Q727',
 'Q23482',
 'Q1435',
 'Q1748',
 'Q8818',
 'Q31487',
 'Q1794',
 'Q10305',
 'Q1524',
 'Q2656',
 'Q1757',
 'Q34370',
 'Q1022',
 'Q1773',
 'Q1718',
 'Q216',
 'Q2079',
 'Q1761',
 'Q1770']

In [35]:
import requests

# request the GeoDB Cities API with a city id
url = "https://wft-geo-db.p.rapidapi.com/v1/geo/cities/Q90"

headers = {
    'x-rapidapi-host': "wft-geo-db.p.rapidapi.com",
    'x-rapidapi-key': geo_db_key
    }

response = requests.request("GET", url, headers=headers)

JSON(response.json())

<IPython.core.display.JSON object>

In [13]:
import time

# request header for the request on the GeoDB Cities API
headers = {
    'x-rapidapi-host': "wft-geo-db.p.rapidapi.com",
    'x-rapidapi-key': geo_db_key
    }  

# get data for all cities in the city id list (list_cityid) from the GeoDB Cities API
def get_city_data(cityid):
    time.sleep(2)
    city_url = f"https://wft-geo-db.p.rapidapi.com/v1/geo/cities/{cityid}"
    city_response = requests.request("GET", city_url, headers=headers)
    city_data = city_response.json()
    return{
        #'wikidata_id' : city_data['data']['wikiDataId'],
        'city' : city_data['data']['city'],
        'country': city_data['data']['country'],
        'country_code': city_data['data']['countryCode'],
        'population': city_data['data']['population'],
        'elevation_meters': city_data['data']['elevationMeters'],
        'latitude': city_data['data']['latitude'],
        'longitude': city_data['data']['longitude']
    }
    

#create a DataFrame with thepopulation data for the cities
city_pop2 = pd.DataFrame([get_city_data(city) for city in list_cityid])

In [14]:
# add a reference column with city + country code to connect database tables
city_pop2 = city_pop2.assign(municipality_iso_country = lambda x: x['city'] + ',' + x['country_code'] )
city_pop2.head()

,city,country,country_code,population,elevation_meters,latitude,longitude,municipality_iso_country
0,Berlin,Germany,DE,3644826,NaN,52.516667,13.383333,"Berlin,DE"
1,Paris,France,FR,2175601,28.0,48.856944,2.351389,"Paris,FR"
2,Barcelona,Spain,ES,1664182,12.0,41.382500,2.176944,"Barcelona,ES"
3,Madrid,Spain,ES,3334730,667.0,40.418889,-3.691944,"Madrid,ES"
4,Istanbul,Turkey,TR,15462452,100.0,41.010000,28.960278,"Istanbul,TR"


## Get Airport Data from csv File

from: http://www.lsv.fr/~sirangel/teaching/dataset/index.html

Name:	Name of airport. May or may not contain the City name. \
City:	Main city served by airport. May be spelled differently from Name. \
Country:	Country or territory where airport is located. \
IATA:	3-letter IATA code (identifier). \
ICAO:	4-letter ICAO code. \
Latitude:	Decimal degrees, usually to six significant digits. Negative is South, positive is North. \
Longitude:	Decimal degrees, usually to six significant digits. Negative is West, positive is East. \
Altitude:	In feet. \
Timezone:	Hours offset from UTC. - dropped \
DST:	Daylight savings time One of E (Europe), A (US/Canada), S (South America), O (Australia), Z (New Zealand), N (None) or U (Unknown). - dropped


In [22]:
#read and process airports data
airports = (
    pd.read_csv('airports_joan.csv', sep=',')
    .query('type == "large_airport"')
    .filter(['name', 'latitude_deg', 'longitude_deg', 'elevation_ft', 'iso_country','iso_region', 'municipality','gps_code','iata_code'])
    .rename(columns={'gps_code':'icao_code', 'name': 'airport_name'})
    .assign(municipality_iso_country = lambda x: x['municipality'] + ',' + x['iso_country'])
)

In [23]:
airports.head()

,airport_name,latitude_deg,longitude_deg,elevation_ft,iso_country,iso_region,municipality,icao_code,iata_code,municipality_iso_country
10890,Honiara International Airport,-9.428000,160.054993,28.0,SB,SB-CT,Honiara,AGGH,HIR,"Honiara,SB"
12461,Port Moresby Jacksons International Airport,-9.443380,147.220001,146.0,PG,PG-NCD,Port Moresby,AYPY,POM,"Port Moresby,PG"
12981,Keflavik International Airport,63.985001,-22.605600,171.0,IS,IS-2,Reykjavík,BIKF,KEF,"Reykjavík,IS"
13028,Priština Adem Jashari International Airport,42.572800,21.035801,1789.0,XK,XK-01,Prishtina,BKPR,PRN,"Prishtina,XK"
17254,Guodu Air Base,36.001741,117.632010,NaN,CN,CN-37,"Xintai, Tai'an",NaN,NaN,"Xintai, Tai'an,CN"


In [24]:
airports.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 624 entries, 10890 to 68323
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   airport_name              624 non-null    object 
 1   latitude_deg              624 non-null    float64
 2   longitude_deg             624 non-null    float64
 3   elevation_ft              622 non-null    float64
 4   iso_country               623 non-null    object 
 5   iso_region                624 non-null    object 
 6   municipality              619 non-null    object 
 7   icao_code                 619 non-null    object 
 8   iata_code                 617 non-null    object 
 9   municipality_iso_country  618 non-null    object 
dtypes: float64(3), object(7)
memory usage: 53.6+ KB


In [25]:
airports['icao_code'].nunique()

619

In [26]:
#dropped the duplicated 'icao' values, basically the ones with \N, where icao was not available
airports.drop_duplicates(subset='icao_code', inplace=True)

## Transfer the data into the local MySQL database

In [35]:
pip install awswrangler

In [7]:
import awswrangler as wr
import pandas as pd
import sqlalchemy
import numpy as np

In [8]:
# connection details for the local mysql database
schema = "gans"
host = "127.0.0.1"
user = "root"
password = mysql_pw
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [52]:
city_pop2 = city_pop2.replace({np.nan}, 'unknown')

In [53]:
city_pop2.to_sql('city_pop', if_exists = 'append', con = con, index=False)

In [55]:
pd.read_sql(
    sql = """
        select * from city_pop
        where elevation_meters > 100
    """,
    con = con
)

,city,country,country_code,population,elevation_meters,latitude,longitude,municipality_iso_country
0,Madrid,Spain,ES,3334730,667,40.418889,-3.691944,"Madrid,ES"
1,Moscow,Russia,RU,12655050,156,55.755833,37.617778,"Moscow,RU"
2,Kyiv,Ukraine,UA,2963199,179,50.450000,30.523611,"Kyiv,UA"
3,Minsk,Belarus,BY,2009786,280,53.900000,27.566670,"Minsk,BY"
4,Vienna,Austria,AT,1911191,171,48.208330,16.373064,"Vienna,AT"
5,Warsaw,Poland,PL,1790658,103,52.216667,21.033333,"Warsaw,PL"
6,Budapest,Hungary,HU,1723836,102,47.498333,19.040833,"Budapest,HU"
7,Munich,Germany,DE,1471508,519,48.134520,11.571000,"Munich,DE"
8,Milan,Italy,IT,1366180,120,45.466944,9.190000,"Milan,IT"
9,Belgrade,Serbia,RS,1378682,117,44.816667,20.466667,"Belgrade,RS"


In [60]:
airports = airports.replace({np.nan}, 'unknown')

In [61]:
airports.head()

,airport_name,latitude_deg,longitude_deg,elevation_ft,iso_country,iso_region,municipality,icao_code,iata_code,municipality_iso_country
10890,Honiara International Airport,-9.428000,160.054993,28.0,SB,SB-CT,Honiara,AGGH,HIR,"Honiara,SB"
12461,Port Moresby Jacksons International Airport,-9.443380,147.220001,146.0,PG,PG-NCD,Port Moresby,AYPY,POM,"Port Moresby,PG"
12981,Keflavik International Airport,63.985001,-22.605600,171.0,IS,IS-2,Reykjavík,BIKF,KEF,"Reykjavík,IS"
13028,Priština Adem Jashari International Airport,42.572800,21.035801,1789.0,XK,XK-01,Prishtina,BKPR,PRN,"Prishtina,XK"
17254,Guodu Air Base,36.001741,117.632010,unknown,CN,CN-37,"Xintai, Tai'an",unknown,unknown,"Xintai, Tai'an,CN"


In [62]:
airports.to_sql('airport', if_exists = 'append', con = con, index = False)

In [63]:
weather_data.head()

,weather_datetime,main_weather,weather_detail,temperature,feels_like,humidity,wind,prob_perc,rain_qty,snow,municipality_iso_country
0,2021-11-30 12:00:00,Rain,light rain,3.29,-2.07,90,8.05,1.00,0.61,0.0,"Berlin,DE"
1,2021-11-30 15:00:00,Rain,light rain,4.22,-1.30,74,9.49,0.35,0.28,0.0,"Berlin,DE"
2,2021-11-30 18:00:00,Clouds,overcast clouds,4.45,-0.94,67,9.30,0.16,0.00,0.0,"Berlin,DE"
3,2021-11-30 21:00:00,Clouds,scattered clouds,2.68,-2.18,75,6.30,0.01,0.00,0.0,"Berlin,DE"
4,2021-12-01 00:00:00,Clouds,broken clouds,2.14,-0.63,88,2.65,0.00,0.00,0.0,"Berlin,DE"


In [10]:
weather_data = weather_data.assign(weather_datetime = lambda x: pd.to_datetime(x['weather_datetime']))
weather_data.to_sql('weather_data', if_exists = 'append', con = con, index=False)

## Transfer the data into the cloud - AWS RDS database

In [30]:
import awswrangler as wr
import pandas as pd
import sqlalchemy
import numpy as np

In [31]:
# connection details for the cloud AWS RDS database
schema = "gans"
host = aws_db
user = "admin"
password = aws_mysql_pw
port = 3306
con2 = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [36]:
city_pop2 = city_pop2.replace({np.nan}, 'unknown')

In [37]:
city_pop2.to_sql('city_pop', if_exists = 'append', con = con2, index=False)

In [38]:
airports = airports.replace({np.nan}, 'unknown')

In [39]:
airports.to_sql('airport', if_exists = 'append', con = con2, index = False)

IntegrityError: (pymysql.err.IntegrityError) (1452, 'Cannot add or update a child row: a foreign key constraint fails (`gans`.`airport`, CONSTRAINT `airport_ibfk_1` FOREIGN KEY (`municipality_iso_country`) REFERENCES `city_pop` (`municipality_iso_country`))')
[SQL: INSERT INTO airport (airport_name, latitude_deg, longitude_deg, elevation_ft, iso_country, iso_region, municipality, icao_code, iata_code, municipality_iso_country) VALUES (%(airport_name)s, %(latitude_deg)s, %(longitude_deg)s, %(elevation_ft)s, %(iso_country)s, %(iso_region)s, %(municipality)s, %(icao_code)s, %(iata_code)s, %(municipality_iso_country)s)]
[parameters: ({'airport_name': 'Honiara International Airport', 'latitude_deg': -9.428, 'longitude_deg': 160.054993, 'elevation_ft': 28.0, 'iso_country': 'SB', 'iso_region': 'SB-CT', 'municipality': 'Honiara', 'icao_code': 'AGGH', 'iata_code': 'HIR', 'municipality_iso_country': 'Honiara,SB'}, {'airport_name': 'Port Moresby Jacksons International Airport', 'latitude_deg': -9

Flights data and weather data will be pushed to the database from the automated AWS Lambda functions. 